## Using tsv file + map() and filter()
Use spark context to read a text file "line by line"

In [ ]:
import findspark

from IPython.display import display
from pyspark.sql import SparkSession

findspark.init("/usr/local/spark-2.3.0-bin-hadoop2.7/")

spark = SparkSession.builder.appName("basics").getOrCreate()
test_file = "/Users/asapehrsson/dev/learn/intro_to_hadoop_and_mapreduce/data/purchases_sample.tsv"

### Read file, store is key, sales is value. Sort by key (even though we don't use it)

In [4]:
test_data = spark.sparkContext.textFile(test_file) \
    .map(lambda line: line.split('\t')) \
    .filter(lambda line: len(line) == 6) \
    .map(lambda line: (line[2], line[4])) \
    .sortByKey() \
    .collect()

print("Number of entries:", len(test_data))


Number of entries: 100


### Read file, store is key, sales is value. Calculate total sales per store.

In [5]:
import tabulate
from IPython.display import HTML

test_data = spark.sparkContext.textFile(test_file) \
    .map(lambda line: line.split('\t')) \
    .filter(lambda line: len(line) == 6) \
    .map(lambda line: (line[2], line[4])) \
    .reduceByKey(lambda a, b: round(float(a) + float(b), 2)) \
    .collect()

print("Number of entries:", len(test_data))

#print(test_data)


Number of entries: 60


### Read file, store is key, sales is value. Calculate total sales per store. Print top three

In [64]:
test_data = spark.sparkContext.textFile(test_file) \
    .map(lambda line: line.split('\t')) \
    .filter(lambda line: len(line) == 6) \
    .map(lambda line: (line[2], line[4])) \
    .reduceByKey(lambda a, b: round(float(a) + float(b), 2)) \
    .sortBy(lambda tuple: float(tuple[1]), ascending=False) \
    .take(3)

display(HTML(tabulate.tabulate(test_data, tablefmt='html')))


Chula Vista,1118.65
Portland,1037.1
Gilbert,969.41
